https://adventofcode.com/2022/day/20

In [1]:
from dataclasses import dataclass, field
from typing import Optional, Self
from tqdm.notebook import tqdm

@dataclass
class Node():
    value: int
#     index: int
    left: Optional[int]=None
    right: Optional[int]=None
    is_head: bool=False
    to_self: Optional[int]=None
        
    def get_left(self,
                 n: int=1) -> Self:
        if n==0:
            return self
        to_return = self.left
        n -= 1
        while n:
            to_return = to_return.left
            if to_return != self:
                n -= 1
        return to_return
        
    def get_right(self,
                  n: int=1) -> Self:
        if n==0:
            return self
        to_return = self.right
        n -= 1
        while n:
            to_return = to_return.right
            if to_return != self:
                n -= 1
        return to_return
        
    def __repr__(self) -> str:
        return f"Node(value={self.value})"

    @property
    def loc(self) -> int:
        loc = 0
        p = self
        while not p.is_head:
            loc += 1
            p = p.get_left()
        return loc
    
    def move_between(self,
                     new_left: Self,
                     new_right: Self) -> None:
        assert (new_left.right == new_right) and (new_right.left == new_left)
        old_left = self.left
        old_right = self.right
        self.right = new_right
        self.left = new_left
        new_left.right = self
        new_right.left = self
        old_left.right = old_right
        old_right.left = old_left
        if self.is_head:
            self.is_head = False
            old_right.is_head = True
        
    def move_right(self,
                   n: int=1) -> None:
        if self.to_self:
            n = n%self.to_self
        if n==0:
            return
        destination = self.get_right(n)
        new_left = destination
        new_right = destination.get_right()
        self.move_between(new_left, new_right)
        
    def move_left(self,
                  n: int=1) -> None:
        if self.to_self:
            n = n%self.to_self
        if n==0:
            return
        destination = self.get_left(n)
        new_left = destination.get_left()
        new_right = destination
        self.move_between(new_left, new_right)
        
    def move(self,
             n: int) -> None:
        if n > 0:
            self.move_right(n)
        elif n < 0:
            self.move_left(-n)
            
    @property
    def loc(self) -> int:
        if self.is_head:
            return 0
        n = 1
        node = self.get_left()
        while not node.is_head:
            n += 1
            node = node.get_left()
        return n
        

@dataclass
class File():
    nodes: list[Node]
        
    def __post_init__(self):
        self.nodes[0].is_head = True

    @property
    def head(self):
        return next(n for n in self.nodes if n.is_head)
    
    @classmethod
    def from_nodes(cls,
                   nodes: list[Node]) -> Self:
        for i, (n_l, n_r) in enumerate(zip(nodes[:-1], nodes[1:])):
            n_l.right = n_r
            n_r.left = n_l
        nodes[-1].right = nodes[0]
        nodes[0].left = nodes[-1]
        for n in nodes:
            n.to_self = len(nodes)-1
            # subtract 1, because if we complete a full cycle we don't count
            # the number being moved.
        return cls(nodes)
    
    def as_list_of_values(self) -> list[int]:
        return [n.value for n in self.as_list()]
    
    def as_list(self) -> list[Node]:
        to_return = [self.head]
        node = self.head.get_right()
        while node != self.head:
            to_return.append(node)
            node = node.get_right()
        return to_return
    
    def __getitem__(self,
                    i: int) -> Node:
        return self.as_list()[i%len(self.nodes)]
    
    def __len__(self) -> int:
        return len(self.numbers)
    
    def node_by_value(self,
                      value: int) -> Node:
        # Warning: this can only be used for the unique node with value 0 in the real data
        return next(n for n in self.nodes if n.value == value)
    
def read_data(filename: str) -> list[int]:
    with open(filename, 'r') as f:
        data = list(map(int, f.readlines()))
    return data

def solve_part_1(filename: str) -> int:
    data = read_data(filename)
    nodes = [Node(i) for i in data]
    file = File.from_nodes(nodes)
    for node in tqdm(file.nodes):
        node.move(node.value)
    loc0 = file.node_by_value(0).loc
    return sum(file[loc0+i].value for i in (1000, 2000, 3000))

def solve_part_2(filename: str, decryption_key=811589153) -> int:
    data = read_data(filename)
    nodes = [Node(i * decryption_key) for i in data]
    file = File.from_nodes(nodes)
    for i in tqdm(range(10)):
        for node in tqdm(file.nodes, leave=False):
            node.move(node.value)
    loc0 = file.node_by_value(0).loc
    return sum(file[loc0+i].value for i in (1000, 2000, 3000))

In [2]:
filename = "../example_data/day20_example_data.txt"
solve_part_1(filename)

  0%|          | 0/7 [00:00<?, ?it/s]

3

In [3]:
filename = "../data/day20_data.txt"
solve_part_1(filename)

  0%|          | 0/5000 [00:00<?, ?it/s]

4426

In [4]:
filename = "../example_data/day20_example_data.txt"
solve_part_2(filename)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

1623178306

In [5]:
filename = "../data/day20_data.txt"
solve_part_2(filename)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

8119137886612